# IDR to CNY, CAD, and MYR currency model

In [1]:
%run helper_functions_and_libraries.py
logging.set_verbosity(logging.ERROR)

In [2]:
df = pd.read_csv('currency_data_10_years.csv', parse_dates=['date'])

In [3]:
display(df.head())

,date,EUR,USD,JPY,GBP,SGD,AUD,CNY,CAD,MYR,RUB
0,2013-05-01,12816.661099,9721.95011,99.794970,15125.789759,7884.025975,9991.932116,1573.729510,9637.238781,3196.705985,311.579294
1,2013-05-02,12696.355896,9727.31307,99.371194,15096.787469,7878.089194,9976.629029,1576.636811,9620.049518,3193.114711,311.787545
2,2013-05-03,12735.428571,9726.93828,98.682506,15125.784565,7879.444847,10024.671009,1576.977359,9628.439345,3201.788787,312.272389
3,2013-05-04,12758.622548,9729.21521,98.372041,15148.170100,7886.016560,10037.051582,1577.842772,9643.293465,3204.827462,313.177271
4,2013-05-05,12719.240307,9697.65758,97.852512,15097.625177,7859.545641,9999.337595,1572.747845,9623.460698,3195.832412,312.291932


In [4]:
df.isnull().sum()

date    0
EUR     0
USD     0
JPY     0
GBP     0
SGD     0
AUD     0
CNY     0
CAD     0
MYR     0
RUB     0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df.describe()

,EUR,USD,JPY,GBP,SGD,AUD,CNY,CAD,MYR,RUB
count,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000
mean,15734.162882,13626.586201,121.401374,18653.481260,10082.717169,10260.649080,2069.300203,10754.881120,3404.138908,231.530939
std,1028.194122,1215.902129,10.240113,1282.776928,714.681696,449.989173,140.321208,530.164824,173.826533,46.941534
min,12502.789487,9697.657580,94.607603,14719.776369,7717.296664,8937.081757,1572.747845,9357.856739,2975.634017,102.737302
25%,15022.483609,13190.926920,113.522422,17928.531116,9568.972790,9936.471898,1980.528819,10411.948518,3302.096195,201.594311
50%,15801.743633,13912.643820,120.609803,18781.087788,10247.419202,10276.735571,2091.106976,10746.358863,3423.539036,219.833077
75%,16463.962255,14384.106330,129.598188,19469.502233,10588.733799,10609.276367,2177.005363,11165.293022,3529.477813,237.545901
max,18035.467645,16644.718800,153.504275,22488.120204,11718.909446,11396.359940,2346.805612,12314.949757,3819.207831,384.795234


In [7]:
SPLIT_TIME = 2922
WINDOW_SIZE = 30
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

## CNY

In [ ]:
cny_df = df[['date','CNY']]

In [ ]:
display(cny_df.head())

In [ ]:
times, series = parse_data_from_df(cny_df, 'CNY')

In [ ]:
plt.figure(figsize=(8, 4))
plot_series(times, series, title="CNY/IDR Currency", xlabel="Days", ylabel="CNY/IDR")

In [ ]:
# Split the dataset
time_train, series_train, time_valid, series_valid = train_val_split(times, series, SPLIT_TIME, None)

In [ ]:
train_set = windowed_dataset(series_train, 
                             window_size=WINDOW_SIZE, 
                             batch_size=BATCH_SIZE, 
                             shuffle_buffer=SHUFFLE_BUFFER_SIZE)

In [ ]:
def create_model():
    
    # Build the model
    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, 
                               strides=1, 
                               padding="causal", 
                               activation="relu", 
                               input_shape=[WINDOW_SIZE, 1]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64), 
        
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1),
        
        tf.keras.layers.Lambda(lambda x: x * 400)
    ])

    return model

In [ ]:
# Test your uncompiled model
model = create_model()

# Print the model summary
model.summary()

# Check model compatibility
try:
    model.predict(train_set)
except:
    print("Your current architecture is incompatible with the windowed dataset, try adjusting it.")
else:
    print("Your current architecture is compatible with the windowed dataset! :)")

In [ ]:
# Get initial weights
init_weights = model.get_weights()

# Reset states generated by Keras
tf.keras.backend.clear_session()

# Reset the weights
model.set_weights(init_weights)

In [ ]:
def adjust_learning_rate(dataset, model_to_train):
    
    model = model_to_train
    
    # Set the learning rate scheduler
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))
    
    # Select optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-8)
    # optimizer = tf.keras.optimizers.SGD(momentum=0.9)
    
    # Compile the model passing in the appropriate loss
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=optimizer, 
                  metrics=["mae"])
    
    # Train the model
    history = model.fit(dataset, epochs=100, callbacks=[lr_schedule])
    
    return history

In [ ]:
# Run the training with dynamic LR
lr_history = adjust_learning_rate(train_set, model)

In [ ]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

# Reset the weights
model.set_weights(init_weights)

In [ ]:
def compile_model(dataset, model_to_train):
    
    model = model_to_train

    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-8)

    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=optimizer,
                  metrics=["mae"])

    history = model.fit(dataset, epochs=100)   

    return history

In [ ]:
history = compile_model(train_set, model)

In [ ]:
visualize_mae_loss(history)

In [ ]:
rnn_forecast_valid = evaluate_forecast(model, times, series, time_valid, series_valid,
                      SPLIT_TIME, -1, WINDOW_SIZE, BATCH_SIZE)

In [ ]:
compute_metrics(series_valid, rnn_forecast_valid)

In [ ]:
print(f'2 last Actual Price = {series_valid[-2:]}, Predicted Price = {rnn_forecast_valid[-2:]}')

In [ ]:
into_future = 100

future_forecast = make_future_forecast(values=series,
                                       model=model,
                                       into_future=into_future,
                                       window_size=WINDOW_SIZE)

In [ ]:
start_future = times[-1] + 1
future_time = np.arange(start_future, start_future + into_future)

# Insert last timestep/final price into next time steps and future forecasts so the plot connects
future_time = np.insert(future_time, 0, times[-1])
future_forecast = np.insert(future_forecast, 0, series[-1])

In [ ]:
plt.figure(figsize=(15,7))
plot_future_forecast(times, series, start=3600, format="-", label="Actual CNY/IDR", xlabel="Days", ylabel="CNY/IDR")
plot_future_forecast(future_time, future_forecast, format="-", label="Predicted CNY/IDR", xlabel="Days", ylabel="CNY/IDR")